In [1]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import json
from typing import Dict, List
from pathlib import Path


# Motivation (TODO)

# Function Definitions

In [2]:
def set_NaN_to_None(x):
    return 'None' if pd.isna(x) else x

def generate_AB_tasks(inputs: pd.DataFrame, outputs_a: pd.DataFrame, outputs_b: pd.DataFrame):
    ids = set(outputs_a.index).intersection(set(outputs_b.index))
    tasks = []
    for id in ids:
        input_row = inputs.loc[id]
        row_a = outputs_a.loc[id]
        row_b = outputs_b.loc[id]
    
        option_a = set_NaN_to_None(row_a.values[0])
        option_b = set_NaN_to_None(row_b.values[0])
        
        task_dict = {
            "id": id,
            "data": {
                "input": input_row,
                "option_A": option_a,
                "option_B": option_b,
            }
        }
        tasks.append(task_dict)
    return tasks

def serialize_tasks(tasks: List[Dict], path: Path):
    json_data = json.dumps(tasks)
    path = Path(path)
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, 'w') as f:
        f.write(json_data)
    print(f'Saved to {path}')


```
<View>
  <Header value="Question"/>
  <Text name="question_id" value="Question ID: $question_id"/>
  <Text name="question" value="$question"/>
  <Header value="Options"/>
  <Text name="options" value="$options"/>
  <Header value="Answer(s)"/>
  <Text name="answer" value="$answer"/>
  <Header value="-----------------"/>
  <Header value="Hints"/>
  <View style="display: flex;">
    <View>
      <Text name="space1" value=" "/>
      <Text name="hint_a" value="A) $option_A"/>
      <Text name="space2" value=" "/>
      <Text name="hint_b" value="B) $option_B"/>
      <Text name="space3" value=" "/>
    </View>
  </View>
  <Header value="Which hint is the best?"/>
    <Choices name="resp" toName="question" required="true">
      <Choice value="Hint A"/>
      <Choice value="Hint B"/>
      <Choice value="I cannot assess"/>
      <Choice value="The hints are basically the same"/>
    </Choices>
</View>
```

# Generate Data

# Model comparison

In [ ]:
hints_dir = 'azin/v3'
hints_name_1 = 'INS_A_V1_GPT4_40'
hints_name_2 = 'INS_A_V2_GPT4_40'
hints_path_1 = Path(f'../data/hints/{hints_dir}/{hints_name_1}.csv')
hints_path_2 = Path(f'../data/hints/{hints_dir}/{hints_name_2}.csv')

AB_tasks_path = Path(f'../data/label_studio/{hints_dir}/{hints_name_1}-vs-{hints_name_2}.json')

hints_df_a = pd.read_csv(hints_path_1)[['question_ptr_id', 'ai_hint']]
hints_df_b = pd.read_csv(hints_path_2)[['question_ptr_id', 'ai_hint']]

questions_df = pd.read_csv('../data/courses/v1/TestSubjects.csv')
questions_df.set_index('question_ptr_id', inplace=True)
hints_df_a.set_index('question_ptr_id', inplace=True)
hints_df_b.set_index('question_ptr_id', inplace=True)
print(AB_tasks_path)
